In [1]:
!pip install mysql-connector-python

You should consider upgrading via the 'c:\users\anike\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


### Import Standard Libraries

In [2]:
import mysql
from mysql import connector
import random
import string
import pandas as pd
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

creds = {'host': 'localhost',
         'user': 'root',
         'password': 'root',
         'database': 'zep'}

### Helper Functions

1. `connect_database()` - Helps you connect to a database.
2. `check_table()` - Checks if a specific table already exists else creates a new one.
3. `show_table()` - Shows all records present in a table.
4. `check_uid()` - Checks if current generated ID already exists.
5. `generate_uid()` - Generates Unique ID of a specific length.

In [3]:
def connect_database(creds):
    """
    Argument: Takes a dictionary containing credentials[host, user, password, database] for connecting to the database.
    Returns: Database connection object
    """
    
    conn = connector.connect(host=creds['host'],
                             user=creds['user'],
                             password=creds['password'],
                             database=creds['database'])
    if conn.is_connected():
        print(f"-------------------------------------------------------------------------------------------------------\n",
              f"Connection to DATABASE:{creds['database']} by USER:{creds['user']} is successfully established")
        
    return conn 


def check_table(conn, table):
    """
    Argument: Takes a connection object and table name and creates table if it doesn't already exists.
    """
    
    cursor = conn.cursor(buffered=True)
    try:
        cursor.execute(f"SELECT *FROM {table};")
        print(f'-------------------------------------------------------------------------------------------------------\n',
              f'TABLE:{table} already exists')
    except mysql.connector.Error as err:
        print('-------------------------------------------------------------------------------------------------------\n',
              'MESSAGE: ', err.msg)
        cursor.execute(f"CREATE TABLE {table}(UID varchar(100), NAME varchar(100));")
        print(f'-------------------------------------------------------------------------------------------------------\n',
              f'MESSAGE: Table {table} created successfully')
    print('-------------------------------------------------------------------------------------------------------')
        
    return cursor

def show_table(conn, table):
    """
    Arguments: Takes a connection object and table name and displays all the records available in the table.
    """
    
    cursor = conn.cursor(buffered=True)
    try:
        cursor.execute(f'SELECT *FROM {table}')
        for r in cursor:
            print(r)
    except mysql.connector.Error as err:
        print(f'ERROR MSG: {err.msg}')
        
def check_uid(uid_list, uid):
    
    flag = False
    
    for i in uid_list:
        if uid == i[0]:
            flag = True
            break
            
    return flag

def generate_uid(length):
    return ''.join([random.choice(string.ascii_letters + string.digits) for i in range(length)])

## Flow of the Application

1. Establish a Database Connection
2. Read all the names
3. Load the certificate template
4. For each name:
    1. Put name on template
    2. Generate UID
    3. Put UID on template
    4. Verify and Insert (UID, NAME) into table

In [4]:
# Establish a connection with respective database
conn_obj = connect_database(creds)

# Check if required table exists or create a new one
mycursor = check_table(conn_obj, 'certificates')

# Read the excel file and create a list of names to iterate through
df = pd.read_excel('/Users/anike/Zep Analytics/Certificate Generator/Names.xlsx')
names = list(df['Names'])

# --------------------------------- Static Variables ------------------

X_BOX = 260
Y_BOX = 567

X = 1013 
Y = 705

X_UID = 1260
Y_UID = 1120
UID_FONT_SIZE = 40

insert_records = ("INSERT INTO certificates "
                  "(UID, Name) "
                  "VALUES (%(UID)s, %(Name)s)")

CERTIFICATE_FOLDER_PATH = 'C:/Users/anike/Zep Analytics/Certificate Generator/Generated Certificates'

# ---------------------------------------------------------------------

for index, name in enumerate(names):
    
    # ================================= CERTIFICATE GENERATION ===========================
    
    temp = cv2.imread("Certificate Template.png")
    UID = generate_uid(16)
    
    # draw rectangle
    cv2.rectangle(temp, (X_BOX, Y_BOX), (X_BOX + 1506, Y_BOX + 185), (256, 256, 256), 2)
    
    # converting opencv's BGR format to PIL's RGB format to load as PIL object
    temp_in_rgb = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(temp_in_rgb)
    draw = ImageDraw.Draw(pil_img)
    
    if len(name) > 29:
        font_size = 110
    elif len(name) > 26:
        font_size = 120
    else:
        font_size = 130
 
    font = ImageFont.truetype("/Windows/Fonts/MTCORSVA.TTF", font_size)
    font2 = ImageFont.truetype('/Windows/Fonts/CALIFI.TTF', UID_FONT_SIZE)
    
    # anchor = 'ms' helps to align the text into the center of the coordinates passed
    draw.text((X, Y), name, font=font, fill=(0, 0, 0), anchor='ms')
    draw.text((X_UID, Y_UID), UID, font=font2, fill=(0, 0, 0))
     
    # converting PIL's RGB format to Opencv's BRG format again    
    cv2_img_processed = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    
    # cv2.putText(temp, name, (X, Y), cv2.FONT_HERSHEY_COMPLEX, get_optimal_font_scale(name, 1506), (0, 0, 0), 1, cv2.LINE_AA)
    cv2.imwrite(f'{CERTIFICATE_FOLDER_PATH}/{name} Certificate.jpg', cv2_img_processed)
    print(' Certificate generating...', index, end='\r')
    
    # ================================= DATABASE INSERTION =================================
    
    # create a dictionary to insert record into the table
    data = {
    'UID': UID,
    'Name': name
    }
    
    # Extract all the existing UID's 
    mycursor.execute(f'SELECT UID FROM certificates')
    existing_uids = mycursor.fetchall()
    
    # check if current generated id is present in existing ID's
    while check_uid(existing_uids, UID) == True:
        UID = generate_uid(16)
    
    # Insert record into the table
    mycursor.execute(insert_records, data)
    
    # Uncomment/comment this to see progress
    # print('Name: ', name, 'with UID: ', UID, 'inserted successfully')

-------------------------------------------------------------------------------------------------------
 Connection to DATABASE:zep by USER:root is successfully established
-------------------------------------------------------------------------------------------------------
 TABLE:certificates already exists
-------------------------------------------------------------------------------------------------------


In [5]:
# df = pd.read_excel('/Users/anike/Zep Analytics/Certificate Generator/Names.xlsx')
# names = list(df['Names'])

# insert_records = ("INSERT INTO certificates "
#                   "(UID, Name) "
#                   "VALUES (%(UID)s, %(Name)s)")

# for name in names:
    
#     # generate unique id
#     UID = generate_uid(16)
    
#     # create a dictionary to insert record into the table
#     data = {
#     'UID': UID,
#     'Name': name
#     }
    
#     # Extract all the existing UID's 
#     mycursor.execute(f'SELECT UID FROM certificates')
#     existing_uids = mycursor.fetchall()
    
#     # check if current generated id is present in existing ID's
#     while check_uid(existing_uids, UID) == True:
#         UID = generate_uid(16)
    
#     # Insert record into the table
#     mycursor.execute(insert_records, data)
    
#     print('Name: ', name, 'with UID: ', UID, 'inserted successfully')

In [6]:
show_table(conn_obj, 'certificates')

('QS5ote3Na3QdtMET', 'Aniket Jalasakare')
('fE1F41mSntpXVR9Y', 'Aadesh Thorat')
('KhYm05QBaM6kXQQK', 'Jay Charole')
('EW27tvCM41yPEZ1m', 'Malhar Jadhav')
('D2HYQkncTaCWKywz', 'Ramakrishnan Gopalswami')
('opDEDjACHUTdfn3C', 'Raamakrishnan Gopalswami')
('rkNLDAJCZNi0b4S8', 'Raamaakrishnan Gopalswami')
('CpcK2MSAi58z6ZOv', 'Raammaakrishnan Gopalswami')
('CI7SMHVG1bj3Jwqe', 'Raammaakkrishnan Gopalswami')
('afXRSU93UKf1fBmV', 'Raammaakkrishnan Gopaalswami')
('686gxrcAimCaECes', 'Raammaakkrishnan Gopaalswaami')
('C693wRXQdKSM1W1a', 'Raammaakkrishnan Gopaalswaammi')


In [7]:
mycursor.execute('SELECT *FROM certificates;')
result = mycursor.fetchall()
len(result)

12

In [8]:
mycursor.execute('DROP TABLE certificates')